In [75]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np

In [121]:
out_df = pd.DataFrame(columns=['Year', 'Total', 'formattedAddress'])

In [122]:
pop = pd.read_csv('population.csv',sep=',')
grouped = pop.groupby('formattedAddress')

In [123]:
for county, group in grouped:
    ada = grouped.get_group(county)
    data = ada[['Year', 'Total']].dropna().astype('float')
    model = smf.ols(formula="Total ~ Year", data=data)
    result = model.fit()
    future_years = pd.DataFrame(np.arange(2016,2026).astype('float'), columns=['Year'])
    sm.add_constant(future_years)
    new_data = pd.DataFrame({ 'Total': result.predict(future_years), 'Year': future_years['Year'],
               'formattedAddress': np.repeat(county, 10)  })
    existing_data = pd.DataFrame({'Total': data['Total'], 'Year': data['Year'],
                              'formattedAddress': np.repeat(county, data['Year'].size)})
    out_df = out_df.append(existing_data)
    out_df = out_df.append(new_data)

In [126]:
out_df.to_csv('predicted_population.csv', columns=['Total','Year','formattedAddress'], index=False)

In [170]:
water = pd.read_csv('water.csv',sep=',')
grouped = water.groupby('FIPS')
out_df = pd.DataFrame(columns=['Water', 'Year', 'FIPS'])
missing = 0
for county, ada in grouped:
    ada = grouped.get_group(county)
    data = ada[['YEAR', 'TO-WTotl']].dropna().astype('float')
    data.columns = ['Year', 'Water']
    model = smf.ols(formula="Water ~ Year", data=data)
    result = model.fit()
    future_years = pd.DataFrame(np.arange(2016,2026).astype('float'), columns=['Year'])
    sm.add_constant(future_years)
    new_data = pd.DataFrame({ 'Water': result.predict(future_years), 'Year': future_years['Year'],
           'FIPS': np.repeat(np.uint32(county), 10)  })
    w = np.array(data['Water'])
    y = np.array(data['Year'])
    nan = np.nan
    if w.size != 3:
        missing = missing + 1
        continue
        
    existing_data = pd.DataFrame({
        'Water': [w[0], nan, nan, nan, nan, w[1], nan, nan, nan, nan, w[2]] },
                             index=[y[0], 2001, 2002, 2003, 2004, y[1], 2006, 2007, 2008, 2009, y[2]])
    existing_data = existing_data.interpolate(method='index')
    existing_data = pd.DataFrame({ 'Water': existing_data['Water'], 'Year': existing_data.index,
                             'FIPS': np.repeat(np.uint32(county), 11) })
    out_df = out_df.append(existing_data)
    out_df = out_df.append(new_data)

In [168]:
missing

698

In [169]:
out_df.to_csv('predicted_water.csv', columns=['Water','Year','FIPS'], index=False)


In [139]:

new_data

,Total,Year,formattedAddress
0,65.616667,2016.0,"Yuma County, AZ, USA"
1,67.446667,2017.0,"Yuma County, AZ, USA"
2,69.276667,2018.0,"Yuma County, AZ, USA"
3,71.106667,2019.0,"Yuma County, AZ, USA"
4,72.936667,2020.0,"Yuma County, AZ, USA"
5,74.766667,2021.0,"Yuma County, AZ, USA"
6,76.596667,2022.0,"Yuma County, AZ, USA"
7,78.426667,2023.0,"Yuma County, AZ, USA"
8,80.256667,2024.0,"Yuma County, AZ, USA"
9,82.086667,2025.0,"Yuma County, AZ, USA"


,FIPS,Water,Year
2000.0,"Yuma County, AZ, USA",37.250,2000.0
2001.0,"Yuma County, AZ, USA",38.532,2001.0
2002.0,"Yuma County, AZ, USA",39.814,2002.0
2003.0,"Yuma County, AZ, USA",41.096,2003.0
2004.0,"Yuma County, AZ, USA",42.378,2004.0
2005.0,"Yuma County, AZ, USA",43.660,2005.0
2006.0,"Yuma County, AZ, USA",46.038,2006.0
2007.0,"Yuma County, AZ, USA",48.416,2007.0
2008.0,"Yuma County, AZ, USA",50.794,2008.0
2009.0,"Yuma County, AZ, USA",53.172,2009.0
